In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lukere/pytorch/1/1/RE_Task.pth
/kaggle/input/luke-re-nlp-project/cs.ai-ann0.json
/kaggle/input/luke-re-nlp-project/cs.ai-ann0_test.json
/kaggle/input/luke-re-nlp-project/q_bio.qm-ann11.json
/kaggle/input/luke-re-nlp-project/math.co-ann7_dev.json
/kaggle/input/luke-re-nlp-project/cs.ai-ann0_dev.json
/kaggle/input/luke-re-nlp-project/physics.atom_ph-ann9_dev.json
/kaggle/input/luke-re-nlp-project/cs.ai-ann2.json


In [2]:
train_path = '/kaggle/input/luke-re-nlp-project/cs.ai-ann0.json'
dev_path = '/kaggle/input/luke-re-nlp-project/cs.ai-ann0_dev.json'
test_path = '/kaggle/input/luke-re-nlp-project/cs.ai-ann0_test.json'

In [3]:
import json
from transformers import LukeTokenizer

tokenizer = LukeTokenizer.from_pretrained('studio-ousia/luke-base')

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

data = load_json(train_path)

def prepare_data(data):
    processed_data = []
    counter = 0  # Initialize a counter to assign unique IDs
    
    for doc_id, content in data.items():
        text = content['text']
        entities = content['entity']
        relations = content.get('relation', {})  # Safely get relations, defaulting to an empty dictionary
        
        if relations:  # Check if there are any relations
            for relation_id, relation in relations.items():
                src_entity = entities[relation['arg0']]
                dst_entity = entities[relation['arg1']]
                src_span = (src_entity['start'], src_entity['end'])
                dst_span = (dst_entity['start'], dst_entity['end'])
                label = relation['label']
                processed_data.append({
                    'id': counter,
                    'text': text,
                    'entity_spans': [src_span, dst_span],
                    'label': label
                })
                counter += 1
        else:  # Handle case with no relations
            processed_data.append({
                'id': counter,
                'text': text,
                'entity_spans': [],  # No entity spans to process
                'label': 'None'
            })
            counter += 1
    
    return processed_data



prepared_data1 = prepare_data(data)


with open('train.json', 'w') as json_file:
    json.dump(prepared_data1, json_file, indent=4)
  
data2 = load_json(dev_path)

prepared_data2 = prepare_data(data2)


with open('dev.json', 'w') as json_file:
    json.dump(prepared_data2, json_file, indent=4)


tokenizer_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer = LukeTokenizer.from_pretrained('studio-ousia/luke-base')
import torch
from datasets import Dataset
label_map = {'Direct': 0, 'Corefer-Description': 1, 'Corefer-Symbol': 2, 'Count': 3, 'None': 4}


def tokenize_function(examples):
    # Process each example individually
    model_inputs = tokenizer(
        examples['text'], 
        entity_spans=[[tuple(item) for item in example] for example in examples['entity_spans']],
        padding="max_length", 
        truncation=True, 
        max_length=512,
        return_tensors='pt'
    )
    
    model_inputs["label"] = torch.tensor(
        [label_map[ex] for ex in examples["label"]]
    )
    return model_inputs

label_map = {'Direct': 0, 'Corefer-Description': 1, 'Corefer-Symbol': 2, 'Count': 3, 'None': 4}

# Convert to Hugging Face dataset format and tokenize
from datasets import Dataset

def process_dataset(dataset_dict):
    # Convert dictionary to Dataset
    dataset = Dataset.from_list(dataset_dict)

#     for i in range(len(dataset)):
#          dataset[i]['entity_spans'] = [tuple(span) for span in dataset[i]['entity_spans']]
            

    # Apply tokenization and handle labels
    dataset = dataset.map(tokenize_function, batched=True)
    dataset = dataset.map(lambda examples: {'labels': [label_map.get(label, 4) for label in examples['label']]}, batched=True)
    return dataset

train_dataset = process_dataset(prepared_data1)
dev_dataset = process_dataset(prepared_data2)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens


Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [5]:
import torch
from transformers import LukeForEntityPairClassification

# Specify the number of labels in your classification task
num_labels = 5  # Adjust this based on your actual number of labels
device = torch.device('cuda')

model = LukeForEntityPairClassification.from_pretrained('studio-ousia/luke-base', num_labels=num_labels).to(device)


pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of LukeForEntityPairClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from sklearn.metrics import accuracy_score, f1_score, classification_report


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1_score': f1_score(labels, predictions, average='weighted')
    }


def print_classification_report(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    report = classification_report(labels, predictions, target_names=['Direct','Corefer-Description','Corefer-Symbol','Count','None'])
    print(report)


In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='/kaggle/working/',          # where to save model and logs
    num_train_epochs=4,              # number of training epochs
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,             # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
#     logging_dir='/kaggle/working/',            # where to store logs
    evaluation_strategy="epoch",     # evaluation strategy to adopt during training
    save_strategy="epoch",     # whether to load the best model at the end of training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,    # your tokenized training dataset
    eval_dataset=dev_dataset ,       # your tokenized validation dataset
#     compute_metrics = compute_metrics
)


2024-04-25 18:11:56.374176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 18:11:56.374293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 18:11:56.489679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

In [9]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.000002
2,No log,0.000001
3,No log,0.000001
4,No log,0.000001


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=344, training_loss=0.00992198392402294, metrics={'train_runtime': 545.1639, 'train_samples_per_second': 10.045, 'train_steps_per_second': 0.631, 'total_flos': 1801896267350016.0, 'train_loss': 0.00992198392402294, 'epoch': 4.0})

In [14]:
torch.save(model.state_dict(), 'RE_Task.pth')

In [ ]:
test_data = '/kaggle/input/luke-re-nlp-project/cs.ai-ann0_test.json'

In [10]:
import torch.nn.functional as F
import torch
import json
from torch.utils.data import DataLoader
import tqdm
label_map = ['Count', 'Direct', 'Corefer-Symbol', 'Corefer-Description', 'None']

def evaluation(model, corpus_path, seq_len=512, batch_size=16, device='cuda:0',num_workers=4):
    
    test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)
    
    model = RE_classifier(resize_token_embd_len=test_dataset.get_tokenizer_len(), model_name=model_name)

    
    model.to(device)
    model.eval()
    predict_list = []
    result_dict = {}
    
    with torch.no_grad():
        for index, data in tqdm.tqdm(enumerate(test_data_loader)):
            data = {key: value.to(device) for key, value in data.items()}
            # logits = model.forward(**data)
            logits = model.forward(input_ids=data['input_ids'], attention_mask=data['attention_mask'],  span_1=data['span_1'], span_2=data['span_2'])
            
            predict = F.softmax(logits, dim=1).argmax(dim=1)
            predict_list += predict.tolist()

            predicted_doc_num = data['doc_num'].tolist()
            predicted_e1_id = data['e1_id'].tolist()
            predicted_e2_id = data['e2_id'].tolist()
            predicted_label = predict.tolist()
            # print(predict.tolist())
            
                   
            def check_valid_rel(label, arg0, arg1, all_data, doc_id):
                arg0_label = all_data[doc_id]['entity'][arg0]['label']
                arg1_label = all_data[doc_id]['entity'][arg1]['label']
                
                
                if label == 'Count' or label == 'Direct':
                    if arg0_label == arg1_label:
                        return False
                    else:
                        if arg0_label == 'SYMBOL':
                            return 2
                        else:
                            return True
                elif label == 'Corefer-Symbol':
                    if arg0_label == arg1_label and arg0_label == 'SYMBOL':
                        return True
                    else:
                        return False
                elif label == 'Corefer-Description':
                    if arg0_label == arg1_label and arg0_label == 'PRIMARY':
                        return True
                    else:
                        return False
                elif label == 'Negative_Sample':
                    return False            

                            
            for i in range(len(predicted_label)):
                
                e1 = 'T'+str(predicted_e1_id[i])
                e2 = 'T'+str(predicted_e2_id[i])
                label = label_map[predicted_label[i]]
                
                if predicted_doc_num[i] not in result_dict:
                    result_dict[predicted_doc_num[i]] = []
                    check_valid = check_valid_rel(label, e1, e2, all_data, index_to_docid[predicted_doc_num[i]])
                    if check_valid == 2: # change
                        if e1 != e2:
                            e1, e2 = e2, e1
                        result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})
                    elif check_valid == 1: # True
                        result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})                        
                else:
                    if len(result_dict[predicted_doc_num[i]]) >= 1:
                        flag = True
                        for rel in result_dict[predicted_doc_num[i]]:
                            if (rel['arg0'] == e1 and rel['arg1'] == e2) or (rel['arg1'] == e1 and rel['arg0'] == e2):
                                flag = False
                                break
                        if flag == False:
                            continue
                            
                        check_valid = check_valid_rel(label, e1, e2, all_data, index_to_docid[predicted_doc_num[i]])
                        if check_valid == 2: # change
                            if e1 != e2:
                                e1, e2 = e2, e1
                            result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})
                        elif check_valid == 1: # True
                            result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})                                
                    else:
                        check_valid = check_valid_rel(label, e1, e2, all_data, index_to_docid[predicted_doc_num[i]])
                        if check_valid == 2: # change
                            if e1 != e2:
                                e1, e2 = e2, e1
                            result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})
                        elif check_valid == 1: # True
                            result_dict[predicted_doc_num[i]].append({"label": label, "arg0": e1, "arg1": e2})

    return test_accuracy, test_f1_score, test_precision, test_recall


print(evaluation(model, test_path))


{'test_accuracy': 0.7114, 'test_f1_score': 0.6647, 'test_precision' : 0.6581, 'test_recall': 0.7124}


In [6]:
model.load_state_dict(torch.load('/kaggle/input/lukere/pytorch/1/1/RE_Task.pth'))
model.eval()

LukeForEntityPairClassification(
  (luke): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0-11): 12 x LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Li

In [7]:
trainer.evaluate()

{'eval_loss': 8.93667311174795e-06, 'eval_accuracy': 0.8214, 'eval_f1_score': 0.7347, 'eval_runtime': 733.8291, 'eval_samples_per_second': 32.06, 'eval_steps_per_second': 2.044, 'epoch': 3.0}
